In [245]:
API_URL = 'http://projects.knmi.nl/klimatologie/daggegevens/getdata_dag.cgi'
API_KEY = 'stns = 235: 280: 260 & vars = VICL: PRCP & byear = 1990 & bmonth = 1 & bday = 1 & eyear = 2020 & emonth = 8 & eday = 18'

import urllib3
import json


with open("raw_data.csv", "w") as dt:
    urllib3.disable_warnings()
    http = urllib3.PoolManager()
    response = http.request('GET', API_URL + '/object',
             fields = {
               'apikey': API_KEY
           })
    dt.write(response.data.decode('utf-8'))

In [246]:
with open("raw_data.csv", "r") as dt:
    file = dt.readlines()

In [247]:
file 

['# BRON: KONINKLIJK NEDERLANDS METEOROLOGISCH INSTITUUT (KNMI)\n',
 '\n',
 '# Opmerking: door stationsverplaatsingen en veranderingen in waarneemmethodieken zijn deze tijdreeksen van dagwaarden mogelijk inhomogeen! Dat betekent dat deze reeks van gemeten waarden niet geschikt is voor trendanalyse. Voor studies naar klimaatverandering verwijzen we naar de gehomogeniseerde reeks maandtemperaturen van De Bilt <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/gehomogeniseerde-reeks-maandtemperaturen-de-bilt> of de Centraal Nederland Temperatuur <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/centraal-nederland-temperatuur-cnt>.\n',
 '\n',
 '# \n',
 '\n',
 '# \n',
 '\n',
 '# STN      LON(east)   LAT(north)     ALT(m)  NAME\n',
 '\n',
 '# 391:         6.197       51.498      19.50  ARCEN\n',
 '\n',
 '# 370:         5.377       51.451      22.60  EINDHOVEN\n',
 '\n',
 '# 331:         4.193       51.480       0.00  THOLEN\n',
 '\n',
 '# 315:         3.998       51.447       0.00  HA

In [248]:
for line in file:
    if '# \n' == line or'\n' == line or '#' in line and 'STN,YYYYMMDD' not in line:
        file.pop(file.index(line))

file

['# STN,YYYYMMDD,DDVEC,FHVEC,   FG,  FHX, FHXH,  FHN, FHNH,  FXX, FXXH,   TG,   TN,  TNH,   TX,  TXH, T10N,T10NH,   SQ,   SP,    Q,   DR,   RH,  RHX, RHXH, EV24,   PG,   PX,  PXH,   PN,  PNH,  VVN, VVNH,  VVX, VVXH,   NG,   UG,   UX,  UXH,   UN,  UNH\n',
 '# \n',
 '  391,20201001,  157,   21,   24,   40,   16,   10,   18,   90,   16,  133,   92,   24,  154,   12,   81,   24,    9,    8,  311,   24,   15,   10,   13,    5,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,   88,   96,   24,   82,   11\n',
 '  391,20201002,   69,   33,   35,   50,    9,   10,    1,  100,    9,  141,   72,    3,  175,   15,   58,    6,   23,   20,  720,    0,   -1,   -1,    1,   12,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,   79,   97,    1,   67,   15\n',
 '  391,20201003,  141,   14,   24,   40,   22,   10,    7,   80,   12,  128,  109,   20,  153,    1,  102,   24,    3,    3,  209,   34,   52,   38,   14,    3,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,   90

In [249]:
import csv
import re
with open('raw_data2.csv', 'w') as dt:
    csvv = csv.writer(dt)
    file[0] = file[0].replace('# ','')
    for row in file:
        row = re.sub(r'(?<=[,])(?=[^\s])', r' ', row)
        csvv.writerow(row.split())
    


In [258]:
import pandas as pd
df = pd.read_csv("raw_data2.csv", error_bad_lines=False)

In [259]:
df.head(2)

STN,  YYYYMMDD, DDVEC, FHVEC,  FG, FHX, FHXH, FHN, FHNH, FXX,  ... VVN,  \
0     #        NaN    NaN    NaN  NaN  NaN   NaN  NaN   NaN  NaN  ...  NaN   
1  391,  20201001,   157,    21,  24,  40,   16,  10,   18,  90,  ...    ,   

  VVNH, VVX, VVXH,  NG,  UG,  UX, UXH,  UN,   UNH  
0   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN  
1     ,    ,     ,    ,  88,  96,  24,  82,  11.0  

[2 rows x 41 columns]

In [260]:
def dfreplace(df, *args, **kwargs):
    s = pd.Series(df.values.flatten())
    s = s.str.replace(*args, **kwargs)
    return pd.DataFrame(s.values.reshape(df.shape), df.index, df.columns)

df = dfreplace(df, ',', '')


In [261]:
df.head()


STN, YYYYMMDD, DDVEC, FHVEC,  FG, FHX, FHXH, FHN, FHNH, FXX,  ... VVN,  \
0    #       NaN    NaN    NaN  NaN  NaN   NaN  NaN   NaN  NaN  ...  NaN   
1  391  20201001    157     21   24   40    16   10    18   90  ...        
2  391  20201002     69     33   35   50     9   10     1  100  ...        
3  391  20201003    141     14   24   40    22   10     7   80  ...        
4  391  20201004    177     34   38   50    10   30     1  130  ...        

  VVNH, VVX, VVXH,  NG,  UG,  UX, UXH,  UN,  UNH  
0   NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1                         88   96   24   82  NaN  
2                         79   97    1   67  NaN  
3                         90   96    7   84  NaN  
4                         77   95    6   60  NaN  

[5 rows x 41 columns]

In [262]:
counting = {}
for value in df['STN,']:
    counting[value] = counting.get(value, 0) + 1
counting

{'#': 1,
 '391': 53,
 '370': 53,
 '331': 53,
 '315': 53,
 '324': 53,
 '375': 53,
 '380': 53,
 '240': 53,
 '286': 53,
 '310': 53,
 '283': 53,
 '280': 53,
 '273': 53,
 '323': 53,
 '249': 53,
 '377': 53,
 '316': 53,
 '313': 53,
 '277': 53,
 '348': 53,
 '308': 53,
 '319': 53,
 '215': 53,
 '278': 53,
 '285': 53,
 '343': 53,
 '225': 53,
 '330': 53,
 '267': 53,
 '269': 53,
 '344': 53,
 '275': 53,
 '235': 53,
 '257': 53,
 '290': 53,
 '350': 53,
 '251': 53,
 '248': 53,
 '279': 53,
 '258': 53,
 '356': 53,
 '209': 53,
 '312': 53,
 '340': 53,
 '260': 53,
 '270': 53,
 '242': 53}